# Body Ablation Search for Ternary Classification

This notebook performs **body ablation analysis** for the ternary classification model.

**Goal**: Find which astrological bodies hurt/help the model by testing all combinations of excluding 0-2 bodies.

## Fixed Parameters (from best previous search):
- **Labeling**: horizon=1, gauss_window=201
- **Features**: helio, orb_multiplier=0.1
- **Model**: depth=9, lr=0.03, sideways_penalty=0.3

## What it does:
1. **Baseline**: No body exclusion
2. **Exclude 1 body**: 11 tests (Sun, Moon, Mercury, ...)
3. **Exclude 2 bodies**: 55 tests (all pairs)

**Total: 67 combinations**

## Output:
- TOP 10 exclusions by UP/DOWN balance
- Which bodies are IMPORTANT (hurt when removed)
- Which bodies are NOISE (help when removed)


In [1]:
# ═══════════════════════════════════════════════════════════════════════════════════
# ENVIRONMENT CHECK & PATH SETUP
# ═══════════════════════════════════════════════════════════════════════════════════
# We need to find the project root directory and add it to Python's path.
# This allows us to import our custom RESEARCH modules from anywhere.
# ═══════════════════════════════════════════════════════════════════════════════════

import sys
from pathlib import Path

# Find project root by looking for RESEARCH folder
PROJECT_ROOT = Path.cwd().resolve()
if not (PROJECT_ROOT / "RESEARCH").exists():
    for parent in PROJECT_ROOT.parents:
        if (parent / "RESEARCH").exists():
            PROJECT_ROOT = parent
            break

if str(PROJECT_ROOT) not in sys.path:
    sys.path.insert(0, str(PROJECT_ROOT))

print(f"Python: {sys.version}")
print(f"PROJECT_ROOT: {PROJECT_ROOT}")


Python: 3.12.12 | packaged by conda-forge | (main, Oct 22 2025, 23:25:55) [GCC 14.3.0]
PROJECT_ROOT: /home/rut/ostrofun


 ## 1. Setup & Configuration

In [2]:
# ═══════════════════════════════════════════════════════════════════════════════════
# IMPORTS
# ═══════════════════════════════════════════════════════════════════════════════════
import itertools
import time
from datetime import datetime
from pathlib import Path
from typing import Dict, List, Any, Optional

import numpy as np
import pandas as pd
# NOTE: tqdm removed - causes output blocking in notebooks
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix, classification_report

# RESEARCH modules
from RESEARCH.config import cfg, PROJECT_ROOT
from RESEARCH.data_loader import load_market_data
from RESEARCH.labeling import create_ternary_labels
from RESEARCH.astro_engine import (
    init_ephemeris,
    calculate_bodies_for_dates_multi,
    calculate_aspects_for_dates,
    calculate_phases_for_dates,
)
from RESEARCH.features import (
    build_full_features,
    merge_features_with_labels,
    get_feature_columns,
)
from RESEARCH.model_training import (
    split_dataset,
    prepare_xy,
    train_xgb_model,
    calc_metrics,
    check_cuda_available,
    compute_stronger_weights,
)
from RESEARCH.cache_utils import (
    save_cache,
    load_cache,
    cache_exists,
    list_cache,
    get_cache_path,
)

print("✓ All modules imported")


✓ All modules imported


In [ ]:
from itertools import combinations

# ═══════════════════════════════════════════════════════════════════════════════════
# BODY ABLATION GRID SEARCH FOR TERNARY MODEL
# ═══════════════════════════════════════════════════════════════════════════════════
# This notebook tests removing combinations of 2 bodies to find which hurt/help performance.
# ═══════════════════════════════════════════════════════════════════════════════════

DATA_START = "2017-11-01"

# ─────────────────────────────────────────────────────────────────────────────────
# FIXED LABELING PARAMS (use best from previous search)
# ─────────────────────────────────────────────────────────────────────────────────
LABEL_GRID = {
    "horizon": [1],
    "gauss_window": [201],
    "gauss_std": [50.0],
}

# ─────────────────────────────────────────────────────────────────────────────────
# FIXED FEATURE PARAMS (use best from previous search)
# ─────────────────────────────────────────────────────────────────────────────────
FEATURE_GRID = {
    "coord_mode": ["both"],
    "orb_multiplier": [0.15],
    "include_phases": [True],
}

# ─────────────────────────────────────────────────────────────────────────────────
# FIXED MODEL PARAMS (use best from previous search)
# ─────────────────────────────────────────────────────────────────────────────────
MODEL_GRID = {
    "max_depth": [9],
    "learning_rate": [0.03],
    "weight_power": [2.0],
    "sideways_penalty": [0.3],
    "n_estimators": [500],
    "subsample": [0.8],
    "colsample_bytree": [0.8],
}

# ─────────────────────────────────────────────────────────────────────────────────
# BODY ABLATION CONFIGURATION
# ─────────────────────────────────────────────────────────────────────────────────
# All available bodies in our astro system
ALL_BODIES = [
    "Sun", "Moon", "Mercury", "Venus", "Mars", 
    "Jupiter", "Saturn", "Uranus", "Neptune", "Pluto", "Node"
]

# Generate all combinations of 2 bodies to EXCLUDE
MAX_EXCLUDE = 2
EXCLUDE_COMBOS = [[]]  # Start with baseline (no exclusion)
for n in range(1, MAX_EXCLUDE + 1):
    for combo in combinations(ALL_BODIES, n):
        EXCLUDE_COMBOS.append(list(combo))

print(f"Total body exclusion combinations: {len(EXCLUDE_COMBOS)}")
print(f"  - Baseline (no exclusion): 1")
print(f"  - Exclude 1 body: {len(ALL_BODIES)}")
print(f"  - Exclude 2 bodies: {len(ALL_BODIES) * (len(ALL_BODIES)-1) // 2}")

# ─────────────────────────────────────────────────────────────────────────────────
# SEARCH SETTINGS
# ─────────────────────────────────────────────────────────────────────────────────
TEST_MODE = False  # Run full ablation search


Total body exclusion combinations: 67
  - Baseline (no exclusion): 1
  - Exclude 1 body: 11
  - Exclude 2 bodies: 55


In [ ]:
# ═══════════════════════════════════════════════════════════════════════════════════
# CALCULATE TOTAL COMBINATIONS
# ═══════════════════════════════════════════════════════════════════════════════════

def count_combinations(grid: Dict[str, List]) -> int:
    """Count total combinations in a parameter grid."""
    total = 1
    for values in grid.values():
        total *= len(values)
    return total

label_combos = count_combinations(LABEL_GRID)
feature_combos = count_combinations(FEATURE_GRID)
model_combos = count_combinations(MODEL_GRID)
total_combos = label_combos * feature_combos * model_combos

print(f"\nGrid Search Statistics:")
print(f"  Label combinations:   {label_combos}")
print(f"  Feature combinations: {feature_combos}")
print(f"  Model combinations:   {model_combos}")
print(f"  ─────────────────────────────")
print(f"  TOTAL COMBINATIONS:   {total_combos}")



Grid Search Statistics:
  Label combinations:   1
  Feature combinations: 1
  Model combinations:   1
  ─────────────────────────────
  TOTAL COMBINATIONS:   1


 ## 2. Load Market Data

In [ ]:
# ═══════════════════════════════════════════════════════════════════════════════════
# LOAD MARKET DATA (WITH CACHING)
# ═══════════════════════════════════════════════════════════════════════════════════

MARKET_CACHE_PARAMS = {
    "subject_id": cfg.active_subject_id,
    "start_date": DATA_START,
}

df_market = load_cache("market", "data", MARKET_CACHE_PARAMS, verbose=True)

if df_market is None:
    print("Loading from database...")
    df_market = load_market_data()
    df_market = df_market[df_market["date"] >= DATA_START].reset_index(drop=True)
    save_cache(df_market, "market", "data", MARKET_CACHE_PARAMS)

print(f"\nMarket data: {len(df_market)} rows")
print(f"Date range: {df_market['date'].min().date()} → {df_market['date'].max().date()}")


📂 Loading from cache: market__data__btc_2017-11-01__8d63944e.parquet

Market data: 3014 rows
Date range: 2017-11-01 → 2026-01-31


 ## 3. Initialize Ephemeris

In [ ]:
# ═══════════════════════════════════════════════════════════════════════════════════
# INITIALIZE ASTRO SETTINGS
# ═══════════════════════════════════════════════════════════════════════════════════

settings = init_ephemeris()
print(f"Bodies: {[b.name for b in settings.bodies]}")

# Check CUDA availability
use_cuda, device = check_cuda_available()
print(f"Device: {device}")


Bodies: ['Sun', 'Moon', 'Mercury', 'Venus', 'Mars', 'Jupiter', 'Saturn', 'Uranus', 'Neptune', 'Pluto', 'TrueNode', 'MeanNode', 'Lilith']
Device: cuda


 ## 4. Grid Search Functions

In [ ]:
# ═══════════════════════════════════════════════════════════════════════════════════
# GRID SEARCH HELPER FUNCTIONS
# ═══════════════════════════════════════════════════════════════════════════════════

def generate_grid_combinations(grid: Dict[str, List]) -> List[Dict[str, Any]]:
    """
    Generate all combinations of parameter values from a grid.
    
    Args:
        grid: Dictionary of parameter_name -> list of values
    
    Returns:
        List of dictionaries, each containing one combination
    
    Example:
        grid = {"a": [1, 2], "b": ["x", "y"]}
        result = [{"a": 1, "b": "x"}, {"a": 1, "b": "y"}, ...]
    """
    keys = list(grid.keys())
    values = list(grid.values())
    
    combinations = []
    for combo in itertools.product(*values):
        combinations.append(dict(zip(keys, combo)))
    
    return combinations


def get_or_compute_labels(
    df_market: pd.DataFrame,
    label_params: Dict,
) -> pd.DataFrame:
    """
    Get labels from cache or compute them.
    
    This is the first step where caching saves the most time,
    since label computation involves detrending which is slow.
    """
    cache_params = {
        "subject_id": cfg.active_subject_id,
        "start_date": DATA_START,
        "type": "ternary",
        **label_params,
    }
    
    # Try cache
    df_labels = load_cache("labels", "ternary", cache_params, verbose=False)
    
    if df_labels is None:
        # Compute
        df_labels, threshold = create_ternary_labels(
            df_market,
            horizon=label_params["horizon"],
            gauss_window=label_params["gauss_window"],
            gauss_std=label_params["gauss_std"],
            balance_classes=True,
            verbose=False,
        )
        save_cache(df_labels, "labels", "ternary", cache_params, verbose=False)
    
    return df_labels


def get_or_compute_features(
    df_market: pd.DataFrame,
    settings,
    feature_params: Dict,
) -> pd.DataFrame:
    """
    Get features from cache or compute them.
    
    Features are the most expensive to compute (bodies + aspects + phases).
    Caching here saves significant time in grid search.
    """
    date_range = f"{df_market['date'].min().date()}_{df_market['date'].max().date()}"
    
    cache_params = {
        "coord_mode": feature_params["coord_mode"],
        "orb_mult": feature_params["orb_multiplier"],
        "include_phases": feature_params["include_phases"],
        "date_range": date_range,
    }
    
    # Try cache for complete feature set
    df_features = load_cache("features", "ternary", cache_params, verbose=False)
    
    if df_features is None:
        # ─────────────────────────────────────────────────────────────────────────
        # Compute bodies (most time-consuming)
        # ─────────────────────────────────────────────────────────────────────────
        bodies_cache_params = {
            "coord_mode": feature_params["coord_mode"],
            "date_range": date_range,
        }
        
        df_bodies = load_cache("astro", "bodies", bodies_cache_params, verbose=False)
        geo_by_date = load_cache("astro", "bodies_geo_dict", bodies_cache_params, verbose=False)
        helio_by_date = load_cache("astro", "bodies_helio_dict", bodies_cache_params, verbose=False)
        
        # If bodies not cached, compute them
        if df_bodies is None:
            df_bodies, geo_by_date, helio_by_date = calculate_bodies_for_dates_multi(
                df_market["date"],
                settings,
                coord_mode=feature_params["coord_mode"],
                progress=False,
            )
            save_cache(df_bodies, "astro", "bodies", bodies_cache_params, verbose=False)
            save_cache(geo_by_date, "astro", "bodies_geo_dict", bodies_cache_params, verbose=False)
            if helio_by_date:
                save_cache(helio_by_date, "astro", "bodies_helio_dict", bodies_cache_params, verbose=False)
        
        # ─────────────────────────────────────────────────────────────────────────
        # Compute aspects (needs geo_by_date)
        # ─────────────────────────────────────────────────────────────────────────
        aspects_cache_params = {
            "orb_mult": feature_params["orb_multiplier"],
            "coord_mode": feature_params["coord_mode"],
            "date_range": date_range,
        }
        
        df_aspects = load_cache("astro", "aspects", aspects_cache_params, verbose=False)
        
        # Only compute if not cached AND we have geo_by_date
        if df_aspects is None:
            if geo_by_date is not None:
                df_aspects = calculate_aspects_for_dates(
                    geo_by_date,
                    settings,
                    orb_mult=feature_params["orb_multiplier"],
                    progress=False,
                )
                save_cache(df_aspects, "astro", "aspects", aspects_cache_params, verbose=False)
            else:
                # Create empty aspects DataFrame if geo_by_date not available
                df_aspects = pd.DataFrame(columns=["date"])
        
        # ─────────────────────────────────────────────────────────────────────────
        # Compute phases
        # ─────────────────────────────────────────────────────────────────────────
        phases_cache_params = {"date_range": date_range}
        
        df_phases = None
        if feature_params["include_phases"]:
            df_phases = load_cache("astro", "phases", phases_cache_params, verbose=False)
            
            if df_phases is None and geo_by_date is not None:
                df_phases = calculate_phases_for_dates(geo_by_date, progress=False)
                save_cache(df_phases, "astro", "phases", phases_cache_params, verbose=False)
        
        # ─────────────────────────────────────────────────────────────────────────
        # Build feature matrix
        # ─────────────────────────────────────────────────────────────────────────
        df_features = build_full_features(
            df_bodies,
            df_aspects,
            df_phases=df_phases,
            include_pair_aspects=True,
        )
        
        save_cache(df_features, "features", "ternary", cache_params, verbose=False)
    
    return df_features


def run_single_experiment(
    df_dataset: pd.DataFrame,
    model_params: Dict,
    device: str,
) -> Dict:
    """
    Run a single training experiment with given parameters.
    
    Returns metrics on test set.
    """
    # ─────────────────────────────────────────────────────────────────────────────
    # Split data
    # ─────────────────────────────────────────────────────────────────────────────
    train_df, val_df, test_df = split_dataset(df_dataset, train_ratio=0.7, val_ratio=0.15)
    
    feature_cols = get_feature_columns(df_dataset)
    X_train, y_train = prepare_xy(train_df, feature_cols)
    X_val, y_val = prepare_xy(val_df, feature_cols)
    X_test, y_test = prepare_xy(test_df, feature_cols)
    
    # ─────────────────────────────────────────────────────────────────────────────
    # Extract weight_power from model_params
    # ─────────────────────────────────────────────────────────────────────────────
    weight_power = model_params.pop("weight_power", 2.0)
    
    # ─────────────────────────────────────────────────────────────────────────────
    # Train model
    # ─────────────────────────────────────────────────────────────────────────────
    model = train_xgb_model(
        X_train, y_train,
        X_val, y_val,
        feature_cols,
        n_classes=3,
        device=device,
        weight_power=weight_power,
        verbose=False,
        **model_params,
    )
    
    # ─────────────────────────────────────────────────────────────────────────────
    # Evaluate on test set
    # ─────────────────────────────────────────────────────────────────────────────
    X_test_scaled = model.scaler.transform(X_test)
    y_proba = model.model.predict_proba(X_test_scaled)
    y_pred = y_proba.argmax(axis=1)
    
    metrics = calc_metrics(y_test, y_pred, labels=[0, 1, 2])
    
    # Add weight_power back for logging
    model_params["weight_power"] = weight_power
    
    return metrics



In [ ]:
# ═══════════════════════════════════════════════════════════════════════════════════
# EVALUATION & VISUALIZATION FUNCTION
# ═══════════════════════════════════════════════════════════════════════════════════
# This function can be called:
#   - BEFORE grid search as baseline
#   - AFTER grid search with best model
# ═══════════════════════════════════════════════════════════════════════════════════

def evaluate_and_visualize_model(
    model,
    X_test: np.ndarray,
    y_test: np.ndarray,
    df_test: pd.DataFrame,
    title: str = "Model Evaluation",
) -> Dict:
    """
    ═══════════════════════════════════════════════════════════════════════════════
    COMPREHENSIVE MODEL EVALUATION WITH VISUALIZATIONS
    ═══════════════════════════════════════════════════════════════════════════════
    
    Displays:
    1. All metrics (accuracy, balanced accuracy, MCC, F1, per-class recalls)
    2. Confusion matrix heatmap
    3. Two price charts with class labels:
       - Top: TRUE labels (from data)
       - Bottom: PREDICTED labels (from model)
    
    Args:
        model: Trained XGBBaseline model
        X_test: Test features (NOT scaled)
        y_test: True labels
        df_test: Test DataFrame with 'date' and 'close' columns
        title: Title for the evaluation
    
    Returns:
        Dictionary with all metrics
    ═══════════════════════════════════════════════════════════════════════════════
    """
    # ─────────────────────────────────────────────────────────────────────────────
    # Get predictions
    # ─────────────────────────────────────────────────────────────────────────────
    X_test_scaled = model.scaler.transform(X_test)
    y_proba = model.model.predict_proba(X_test_scaled)
    y_pred = y_proba.argmax(axis=1)
    
    # ─────────────────────────────────────────────────────────────────────────────
    # Calculate metrics
    # ─────────────────────────────────────────────────────────────────────────────
    metrics = calc_metrics(y_test, y_pred, labels=[0, 1, 2])
    
    CLASS_NAMES = ["DOWN", "SIDEWAYS", "UP"]
    
    print("\n" + "=" * 80)
    print(f"  {title}")
    print("=" * 80)
    
    print("\n📊 METRICS:")
    print(f"  Accuracy:          {metrics['acc']:.4f}")
    print(f"  Balanced Accuracy: {metrics['bal_acc']:.4f}")
    print(f"  MCC:               {metrics['mcc']:.4f}")
    print(f"  F1 Macro:          {metrics['f1_macro']:.4f}")
    
    print("\n📈 PER-CLASS RECALL:")
    print(f"  DOWN (0):     {metrics['recall_down']:.4f}")
    
    # Calculate SIDEWAYS recall manually
    mask_sideways = (y_test == 1)
    recall_sideways = (y_pred[mask_sideways] == 1).sum() / mask_sideways.sum() if mask_sideways.sum() > 0 else 0
    print(f"  SIDEWAYS (1): {recall_sideways:.4f}")
    print(f"  UP (2):       {metrics['recall_up']:.4f}")
    
    print(f"\n🎯 RECALL MIN:  {metrics['recall_min']:.4f} (quality)")
    print(f"   RECALL GAP:  {metrics['recall_gap']:.4f} (balance)")
    
    # ─────────────────────────────────────────────────────────────────────────────
    # Classification report
    # ─────────────────────────────────────────────────────────────────────────────
    print("\n📋 CLASSIFICATION REPORT:")
    print(classification_report(y_test, y_pred, target_names=CLASS_NAMES, zero_division=0))
    
    # ─────────────────────────────────────────────────────────────────────────────
    # Create figure with 3 subplots
    # ─────────────────────────────────────────────────────────────────────────────
    plt.style.use('default')
    fig, axes = plt.subplots(3, 1, figsize=(14, 12), facecolor='white')
    for ax in axes:
        ax.set_facecolor('white')
    
    # ─────────────────────────────────────────────────────────────────────────────
    # Subplot 1: Confusion Matrix
    # ─────────────────────────────────────────────────────────────────────────────
    cm = confusion_matrix(y_test, y_pred, labels=[0, 1, 2])
    sns.heatmap(
        cm, annot=True, fmt="d", cmap="Blues",
        xticklabels=CLASS_NAMES, yticklabels=CLASS_NAMES,
        ax=axes[0]
    )
    axes[0].set_xlabel("Predicted")
    axes[0].set_ylabel("True")
    axes[0].set_title(f"Confusion Matrix | Bal.Acc={metrics['bal_acc']:.3f}, Recall MIN={metrics['recall_min']:.3f}")
    
    # ─────────────────────────────────────────────────────────────────────────────
    # Subplot 2: Price with TRUE labels
    # ─────────────────────────────────────────────────────────────────────────────
    dates = df_test["date"].values
    prices = df_test["close"].values
    
    # Color mapping: DOWN=red, SIDEWAYS=gray, UP=green
    COLORS = {0: "#e63946", 1: "#FFFAFA", 2: "#2d6a4f"}
    
    ax = axes[1]
    ax.plot(dates, prices, color="black", linewidth=1, alpha=0.8)
    
    # Fill background with true class colors
    for i in range(len(dates) - 1):
        ax.axvspan(dates[i], dates[i+1], alpha=0.3, color=COLORS[y_test[i]], linewidth=0)
    
    ax.set_title("TRUE Labels (Ground Truth)")
    ax.set_ylabel("Price")
    ax.grid(True, alpha=0.3)
    
    # Legend
    from matplotlib.patches import Patch
    legend_elements = [
        Patch(facecolor=COLORS[0], alpha=0.5, label="DOWN"),
        Patch(facecolor=COLORS[1], alpha=0.5, label="SIDEWAYS"),
        Patch(facecolor=COLORS[2], alpha=0.5, label="UP"),
    ]
    ax.legend(handles=legend_elements, loc="upper left")
    
    # ─────────────────────────────────────────────────────────────────────────────
    # Subplot 3: Price with PREDICTED labels
    # ─────────────────────────────────────────────────────────────────────────────
    ax = axes[2]
    ax.plot(dates, prices, color="black", linewidth=1, alpha=0.8)
    
    # Fill background with predicted class colors
    for i in range(len(dates) - 1):
        ax.axvspan(dates[i], dates[i+1], alpha=0.3, color=COLORS[y_pred[i]], linewidth=0)
    
    ax.set_title("PREDICTED Labels (Model Output)")
    ax.set_xlabel("Date")
    ax.set_ylabel("Price")
    ax.grid(True, alpha=0.3)
    ax.legend(handles=legend_elements, loc="upper left")
    
    plt.tight_layout()
    plt.suptitle(title, fontsize=14, fontweight="bold", y=1.02)
    plt.show()
    
    return metrics


def train_and_evaluate_full(
    df_dataset: pd.DataFrame,
    model_params: Dict,
    device: str,
    title: str = "Model Evaluation",
    df_market: pd.DataFrame = None,
) -> Dict:
    """
    Convenience function: train model and run full evaluation.
    
    Use this for baseline comparison before grid search
    and for final evaluation after grid search.
    
    Args:
        df_dataset: Dataset with features and labels
        model_params: Model hyperparameters
        device: 'cpu' or 'cuda'
        title: Title for plots
        df_market: Original market data with 'close' column (for visualization)
    """
    # Split data
    train_df, val_df, test_df = split_dataset(df_dataset, train_ratio=0.7, val_ratio=0.15)
    
    # ─────────────────────────────────────────────────────────────────────────────
    # Add 'close' column if missing (needed for price chart)
    # ─────────────────────────────────────────────────────────────────────────────
    if "close" not in test_df.columns and df_market is not None:
        test_df = test_df.merge(
            df_market[["date", "close"]], 
            on="date", 
            how="left"
        )
    elif "close" not in test_df.columns:
        # Create dummy price column if df_market not provided
        test_df = test_df.copy()
        test_df["close"] = 1.0  # Placeholder
    
    feature_cols = get_feature_columns(df_dataset)
    X_train, y_train = prepare_xy(train_df, feature_cols)
    X_val, y_val = prepare_xy(val_df, feature_cols)
    X_test, y_test = prepare_xy(test_df, feature_cols)
    
    # Extract weight_power
    weight_power = model_params.pop("weight_power", 2.0) if "weight_power" in model_params else 2.0
    
    # Train model
    print(f"Training model with weight_power={weight_power}...")
    model = train_xgb_model(
        X_train, y_train,
        X_val, y_val,
        feature_cols,
        n_classes=3,
        device=device,
        weight_power=weight_power,
        verbose=False,
        **model_params,
    )
    
    # Evaluate
    metrics = evaluate_and_visualize_model(model, X_test, y_test, test_df, title)
    
    return metrics, model



 ## 5. Run Grid Search

In [ ]:
# PRE-COMPUTE cell removed - using get_or_compute_features instead
# (it has its own caching)
print("Using cached features approach")


Using cached features approach


In [ ]:
# ═══════════════════════════════════════════════════════════════════════════════════
# SIMPLE FEATURE FILTERING FUNCTION
# ═══════════════════════════════════════════════════════════════════════════════════

def filter_features_by_exclusion(df_features: pd.DataFrame, exclude_bodies: list) -> pd.DataFrame:
    """
    Filter out feature columns related to excluded bodies.
    
    Args:
        df_features: Full features DataFrame
        exclude_bodies: List of body names to exclude (e.g., ['Moon', 'Venus'])
    
    Returns:
        Filtered DataFrame
    """
    if not exclude_bodies:
        return df_features
    
    df_filtered = df_features.copy()
    
    for body in exclude_bodies:
        body_lower = body.lower()
        # Find columns containing body name (case-insensitive)
        cols_to_drop = [c for c in df_filtered.columns 
                        if body_lower in c.lower() and c != 'date' and c != 'target']
        df_filtered = df_filtered.drop(columns=cols_to_drop, errors='ignore')
    
    return df_filtered

print("✓ filter_features_by_exclusion() defined")


✓ filter_features_by_exclusion() defined


In [ ]:
# ═══════════════════════════════════════════════════════════════════════════════════
# RUN BODY ABLATION SEARCH
# ═══════════════════════════════════════════════════════════════════════════════════

# Fixed params
label_params = {
    "horizon": LABEL_GRID["horizon"][0],
    "gauss_window": LABEL_GRID["gauss_window"][0],
    "gauss_std": LABEL_GRID["gauss_std"][0],
}
feature_params = {
    "coord_mode": FEATURE_GRID["coord_mode"][0],
    "orb_multiplier": FEATURE_GRID["orb_multiplier"][0],
    "include_phases": FEATURE_GRID["include_phases"][0],
}
model_params = {
    "max_depth": MODEL_GRID["max_depth"][0],
    "learning_rate": MODEL_GRID["learning_rate"][0],
    "weight_power": MODEL_GRID["weight_power"][0],
    "sideways_penalty": MODEL_GRID["sideways_penalty"][0],
    "n_estimators": MODEL_GRID["n_estimators"][0],
    "subsample": MODEL_GRID["subsample"][0],
    "colsample_bytree": MODEL_GRID["colsample_bytree"][0],
}

total_combos = len(EXCLUDE_COMBOS)
print(f"\nStarting ablation with {total_combos} combinations...")
print(f"Device: {device}")
start_time = time.time()

# ─────────────────────────────────────────────────────────────────────────────
# Pre-compute FULL features (no exclusion) - this is cached
# ─────────────────────────────────────────────────────────────────────────────
print("Computing full features (will be cached)...")
df_features_full = get_or_compute_features(df_market, settings, feature_params)
df_labels = get_or_compute_labels(df_market, label_params)
df_dataset_full = merge_features_with_labels(df_features_full, df_labels)
print(f"Full dataset: {len(df_dataset_full)} samples, {len(get_feature_columns(df_dataset_full))} features")

results = []
best_up_down_min = 0.0
best_exclude = []

for idx, exclude_list in enumerate(EXCLUDE_COMBOS):
    try:
        # ─────────────────────────────────────────────────────────────────────────
        # Filter features by removing excluded body columns
        # ─────────────────────────────────────────────────────────────────────────
        df_dataset = filter_features_by_exclusion(df_dataset_full, exclude_list)
        
        # ─────────────────────────────────────────────────────────────────────────
        # Run experiment
        # ─────────────────────────────────────────────────────────────────────────
        metrics = run_single_experiment(df_dataset, model_params.copy(), device)
        
        up_down_min = min(metrics["recall_down"], metrics["recall_up"])
        up_down_gap = abs(metrics["recall_down"] - metrics["recall_up"])
        
        # Store results
        result = {
            "exclude_bodies": ",".join(exclude_list) if exclude_list else "NONE",
            "n_excluded": len(exclude_list),
            "n_features": len(get_feature_columns(df_dataset)),
            "bal_acc": metrics["bal_acc"],
            "recall_down": metrics["recall_down"],
            "recall_up": metrics["recall_up"],
            "up_down_min": up_down_min,
            "up_down_gap": up_down_gap,
        }
        results.append(result)
        
        # Print progress
        exclude_str = ",".join(exclude_list) if exclude_list else "BASELINE"
        print(
            f"[{idx+1:3d}/{total_combos}] D={metrics['recall_down']:.2f} U={metrics['recall_up']:.2f} "
            f"| ud_min={up_down_min:.2f} | f={len(get_feature_columns(df_dataset)):3d} | {exclude_str}"
        )
        
        if up_down_min > best_up_down_min:
            best_up_down_min = up_down_min
            best_exclude = exclude_list
            print(f"  ⭐ NEW BEST!")
            
    except Exception as e:
        print(f"[{idx+1:3d}/{total_combos}] ⚠️ Error: {e}")

elapsed = time.time() - start_time
print(f"\n✓ Ablation completed in {elapsed/60:.1f} minutes")
print(f"\n🏆 BEST: up_down_min={best_up_down_min:.3f}")
print(f"   Exclude: {best_exclude if best_exclude else 'NONE'}")



Starting ablation with 67 combinations...
Device: cuda
Computing full features (will be cached)...
Merged dataset: 3014 samples (ALL days, forward-filled)
Features: 834
Full dataset: 3014 samples, 834 features
Split: Train=2109, Val=452, Test=453


/mnt/w/WSL/btc/lib/python3.12/site-packages/xgboost/core.py:774: UserWarning: [02:10:21] WARNING: /workspace/src/common/error_msg.cc:62: Falling back to prediction using DMatrix due to mismatched devices. This might lead to higher memory usage and slower performance. XGBoost is running on: cuda:0, while the input data is on: cpu.
Potential solutions:
- Use a data structure that matches the device ordinal in the booster.
- Set the device for booster before call to inplace_predict.

This warning will only be shown once.

  return func(**kwargs)


[  1/67] D=0.61 U=0.00 | ud_min=0.00 | f=834 | BASELINE
Split: Train=2109, Val=452, Test=453
[  2/67] D=0.65 U=0.00 | ud_min=0.00 | f=720 | Sun
Split: Train=2109, Val=452, Test=453
[  3/67] D=0.57 U=0.00 | ud_min=0.00 | f=701 | Moon
Split: Train=2109, Val=452, Test=453
[  4/67] D=0.56 U=0.00 | ud_min=0.00 | f=716 | Mercury
Split: Train=2109, Val=452, Test=453
[  5/67] D=0.62 U=0.00 | ud_min=0.00 | f=716 | Venus
Split: Train=2109, Val=452, Test=453
[  6/67] D=0.55 U=0.00 | ud_min=0.00 | f=702 | Mars
Split: Train=2109, Val=452, Test=453
[  7/67] D=0.56 U=0.00 | ud_min=0.00 | f=714 | Jupiter
Split: Train=2109, Val=452, Test=453
[  8/67] D=0.73 U=0.02 | ud_min=0.02 | f=724 | Saturn
  ⭐ NEW BEST!
Split: Train=2109, Val=452, Test=453
[  9/67] D=0.71 U=0.00 | ud_min=0.00 | f=732 | Uranus
Split: Train=2109, Val=452, Test=453
[ 10/67] D=0.68 U=0.00 | ud_min=0.00 | f=730 | Neptune
Split: Train=2109, Val=452, Test=453
[ 11/67] D=0.59 U=0.04 | ud_min=0.04 | f=732 | Pluto
  ⭐ NEW BEST!
Split: Train

In [ ]:
# ═══════════════════════════════════════════════════════════════════════════════════
# ANALYZE ABLATION RESULTS
# ═══════════════════════════════════════════════════════════════════════════════════

df_results = pd.DataFrame(results)

if not df_results.empty:
    # Sort by UP/DOWN balance
    df_results = df_results.sort_values(
        ["up_down_min", "up_down_gap"], 
        ascending=[False, True]
    ).reset_index(drop=True)
    
    print("\n" + "=" * 80)
    print("🏆 TOP 10 BODY EXCLUSIONS BY UP/DOWN BALANCE")
    print("=" * 80)
    
    display_cols = [
        "exclude_bodies", "n_excluded", "n_features",
        "recall_down", "recall_up", "up_down_min", "up_down_gap", "bal_acc"
    ]
    print(df_results[display_cols].head(10).to_string(index=False))
    
    # ─────────────────────────────────────────────────────────────────────────────
    # Compare baseline vs best
    # ─────────────────────────────────────────────────────────────────────────────
    baseline = df_results[df_results["exclude_bodies"] == "NONE"]
    best = df_results.iloc[0]
    
    if not baseline.empty:
        bl = baseline.iloc[0]
        print("\n" + "=" * 80)
        print("BASELINE vs BEST COMPARISON")
        print("=" * 80)
        print(f"\nBASELINE (no exclusion):")
        print(f"  D={bl['recall_down']:.3f}, U={bl['recall_up']:.3f}, ud_min={bl['up_down_min']:.3f}")
        print(f"\nBEST (exclude: {best['exclude_bodies']}):")
        print(f"  D={best['recall_down']:.3f}, U={best['recall_up']:.3f}, ud_min={best['up_down_min']:.3f}")
        
        improvement = best['up_down_min'] - bl['up_down_min']
        print(f"\n📈 Improvement: {improvement:+.3f} ({improvement/bl['up_down_min']*100:+.1f}%)" if bl['up_down_min'] > 0 else "")
    
    # ─────────────────────────────────────────────────────────────────────────────
    # Which bodies hurt when removed?
    # ─────────────────────────────────────────────────────────────────────────────
    print("\n" + "=" * 80)
    print("SINGLE BODY EXCLUSION ANALYSIS")
    print("=" * 80)
    
    single_exclusions = df_results[df_results["n_excluded"] == 1].copy()
    single_exclusions = single_exclusions.sort_values("up_down_min", ascending=False)
    print("\nBest to exclude (improves model):")
    print(single_exclusions[["exclude_bodies", "recall_down", "recall_up", "up_down_min"]].head(5).to_string(index=False))
    
    print("\nWorst to exclude (hurts model - these are IMPORTANT bodies):")
    print(single_exclusions[["exclude_bodies", "recall_down", "recall_up", "up_down_min"]].tail(5).to_string(index=False))

else:
    print("⚠️ No results to analyze")



🏆 TOP 10 BODY EXCLUSIONS BY UP/DOWN BALANCE
exclude_bodies  n_excluded  n_features  recall_down  recall_up  up_down_min  up_down_gap  bal_acc
   Mars,Uranus           2         610     0.579208   0.104167     0.104167     0.475041 0.365723
    Sun,Saturn           2         620     0.524752   0.083333     0.083333     0.441419 0.358689
 Mercury,Pluto           2         624     0.529703   0.062500     0.062500     0.467203 0.358321
         Pluto           1         732     0.594059   0.041667     0.041667     0.552393 0.374470
   Moon,Uranus           2         609     0.470297   0.020833     0.020833     0.449464 0.344334
    Pluto,Node           2         518     0.579208   0.020833     0.020833     0.558375 0.339587
     Sun,Pluto           2         628     0.589109   0.020833     0.020833     0.568276 0.357665
      Sun,Node           2         510     0.658416   0.020833     0.020833     0.637583 0.352853
        Saturn           1         724     0.732673   0.020833     0.0208

In [ ]:
# ═══════════════════════════════════════════════════════════════════════════════════
# SAVE RESULTS TO CSV
# ═══════════════════════════════════════════════════════════════════════════════════
# SORT BY UP/DOWN BALANCE
if not df_results.empty and "up_down_min" in df_results.columns:
    df_results = df_results.sort_values(
        ["up_down_min", "up_down_gap"], 
        ascending=[False, True]
    ).reset_index(drop=True)
    print(f"✓ Sorted by up_down_min. Best: {df_results['up_down_min'].iloc[0]:.3f}")
    
if not df_results.empty:
    reports_dir = PROJECT_ROOT / "RESEARCH" / "reports"
    reports_dir.mkdir(parents=True, exist_ok=True)
    
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    report_path = reports_dir / f"grid_search_ternary_{timestamp}.csv"
    
    df_results.to_csv(report_path, index=False)
    print(f"\n✓ Results saved to: {report_path}")


✓ Sorted by up_down_min. Best: 0.104

✓ Results saved to: /home/rut/ostrofun/RESEARCH/reports/grid_search_ternary_20260204_021228.csv


In [ ]:
# ═══════════════════════════════════════════════════════════════════════════════════
# BEST BODY EXCLUSION SUMMARY
# ═══════════════════════════════════════════════════════════════════════════════════

if not df_results.empty:
    best = df_results.iloc[0]
    
    print("\n" + "=" * 80)
    print("🏆 BEST BODY EXCLUSION")
    print("=" * 80)
    
    print(f"\nExclude: {best['exclude_bodies']}")
    print(f"Features: {best['n_features']}")
    
    print(f"\nMetrics:")
    print(f"  Recall DOWN:  {best['recall_down']:.4f}")
    print(f"  Recall UP:    {best['recall_up']:.4f}")
    print(f"  UP/DOWN min:  {best['up_down_min']:.4f}")
    print(f"  UP/DOWN gap:  {best['up_down_gap']:.4f}")
    print(f"  Balanced Acc: {best['bal_acc']:.4f}")



🏆 BEST BODY EXCLUSION

Exclude: Mars,Uranus
Features: 610

Metrics:
  Recall DOWN:  0.5792
  Recall UP:    0.1042
  UP/DOWN min:  0.1042
  UP/DOWN gap:  0.4750
  Balanced Acc: 0.3657


In [ ]:
# ═══════════════════════════════════════════════════════════════════════════════════
# SHOW CACHED FILES
# ═══════════════════════════════════════════════════════════════════════════════════

print("\n📁 Cached files after grid search:")
cache_df = list_cache()
if not cache_df.empty:
    print(cache_df[["category", "name", "params", "size_mb"]].to_string(index=False))
    print(f"\nTotal cache size: {cache_df['size_mb'].sum():.2f} MB")
else:
    print("  No cached files found")



📁 Cached files after grid search:
category              name                               params  size_mb
   astro           aspects          2017-11-01_2026_both_orb0.5 0.308278
   astro           aspects         2017-11-01_2026_both_orb0.25 0.181644
   astro           aspects         2017-11-01_2026_both_orb0.15 0.117138
   astro           aspects          2017-11-01_2026_both_orb0.1 0.080457
   astro           aspects         2017-11-01_2026_helio_orb0.5 0.000607
   astro           aspects        2017-11-01_2026_helio_orb0.25 0.000607
   astro           aspects        2017-11-01_2026_helio_orb0.15 0.000607
   astro           aspects         2017-11-01_2026_helio_orb0.1 0.000607
   astro           aspects          2017-11-01_2026_both_orb1.0 0.551617
   astro           aspects           2017-11-01_2026_geo_orb1.0 0.551617
   astro            bodies                2017-11-01_2026_helio 1.081944
   astro            bodies                 2017-11-01_2026_both 2.508477
   astro        

 ## Summary



 Grid search complete! The results show the best parameter combinations for:



 - **Labeling**: horizon, gauss_window (trend smoothing)

 - **Features**: coord_mode (geo/helio), orb_multiplier (aspect strictness)

 - **Model**: max_depth, learning_rate, weight_power (class balance)



 Use the best parameters in `main_pipeline_ternary.py` for production model.

In [ ]:
# ═══════════════════════════════════════════════════════════════════════════════════
# FINAL EVALUATION OF BEST EXCLUSION
# ═══════════════════════════════════════════════════════════════════════════════════

if not df_results.empty:
    best = df_results.iloc[0]
    best_exclude_str = best["exclude_bodies"]
    best_exclude_list = best_exclude_str.split(",") if best_exclude_str != "NONE" else []
    
    print("\n" + "=" * 80)
    print(f"TRAINING BEST MODEL (exclude: {best_exclude_str})")
    print("=" * 80)
    
    best_label_params = {
        "horizon": LABEL_GRID["horizon"][0],
        "gauss_window": LABEL_GRID["gauss_window"][0],
        "gauss_std": LABEL_GRID["gauss_std"][0],
    }
    
    best_model_params = {
        "max_depth": MODEL_GRID["max_depth"][0],
        "learning_rate": MODEL_GRID["learning_rate"][0],
        "n_estimators": MODEL_GRID["n_estimators"][0],
        "weight_power": MODEL_GRID["weight_power"][0],
        "sideways_penalty": MODEL_GRID["sideways_penalty"][0],
    }
    
    print(f"Exclude: {best_exclude_list}")
    
    # ─────────────────────────────────────────────────────────────────────────────
    # Use filter_features_by_exclusion on full dataset
    # ─────────────────────────────────────────────────────────────────────────────
    df_dataset_best = filter_features_by_exclusion(df_dataset_full, best_exclude_list)
    
    print(f"Dataset: {len(df_dataset_best)} samples, {len(get_feature_columns(df_dataset_best))} features")
    
    # Train and evaluate
    final_metrics, final_model = train_and_evaluate_full(
        df_dataset_best,
        best_model_params.copy(),
        device,
        title=f"BEST (exclude: {best_exclude_str})",
        df_market=df_market,
    )
    
    print("\n✓ Evaluation complete!")



TRAINING BEST MODEL (exclude: Mars,Uranus)
Label params: {'horizon': 1, 'gauss_window': 201, 'gauss_std': 50.0}
Exclude: ['Mars', 'Uranus']
Model: {'max_depth': 9, 'learning_rate': 0.03, 'n_estimators': 500, 'weight_power': 2.0, 'sideways_penalty': 0.3}


NameError: name 'compute_features_fast' is not defined

In [ ]:
# ═══════════════════════════════════════════════════════════════════════════════════
# EVALUATE TOP 2 CONFIGURATIONS
# ═══════════════════════════════════════════════════════════════════════════════════

# Sort by UP/DOWN balance
df_best = df_results.sort_values(
    ["up_down_min", "up_down_gap"], 
    ascending=[False, True]
).reset_index(drop=True)

print("TOP 5 by UP/DOWN balance:")
print(df_best[["horizon", "orb_multiplier", "coord_mode", "sideways_penalty", 
               "max_depth", "learning_rate", "recall_down", "recall_up", "up_down_min"]].head())

# Evaluate TOP 2
for rank in range(5):
    best = df_best.iloc[rank]
    
    print(f"\n\n{'='*80}")
    print(f"EVALUATING RANK #{rank+1}")
    print(f"{'='*80}")
    
    best_label_params = {
        "horizon": int(best["horizon"]),
        "gauss_window": int(best["gauss_window"]),
        "gauss_std": float(best.get("gauss_std", 50.0)),
    }
    
    best_feature_params = {
        "coord_mode": best["coord_mode"],
        "orb_multiplier": float(best["orb_multiplier"]),
        "include_phases": True,
    }
    
    best_model_params = {
        "max_depth": int(best["max_depth"]),
        "learning_rate": float(best["learning_rate"]),
        "n_estimators": 500,
        "weight_power": float(best.get("weight_power", 2.0)),
        "sideways_penalty": float(best.get("sideways_penalty", 1.0)),
    }
    
    print(f"Label: {best_label_params}")
    print(f"Feature: {best_feature_params}")
    print(f"Model: {best_model_params}")
    
    # Rebuild dataset
    df_labels_best = get_or_compute_labels(df_market, best_label_params)
    df_features_best = get_or_compute_features(df_market, settings, best_feature_params)
    df_dataset_best = merge_features_with_labels(df_features_best, df_labels_best)
    
    # Train and evaluate
    metrics, model = train_and_evaluate_full(
        df_dataset_best,
        best_model_params.copy(),
        device,
        title=f"RANK #{rank+1} (up_down_min={best['up_down_min']:.3f})",
        df_market=df_market,
    )

KeyError: 'up_down_min'